In [ ]:
!git clone https://github.com/openai/shap-e.git

%cd shap-e
!pip install -e .

Cloning into 'shap-e'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 325 (delta 30), reused 14 (delta 14), pack-reused 278
Receiving objects: 100% (325/325), 11.71 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/shap-e
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-sjp_d_ty/clip_5d4764d14d75447cb48d402b10f33d41
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-sjp_d_ty/clip_5d4764d14d75447cb48d402b10f33d41
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.8 MB/s eta 0:0

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


In [ ]:
%cd shap-e

[Errno 2] No such file or directory: 'shap-e'
/content/shap-e


In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:09<00:00, 100MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
batch_size=1
guidance_scale= 15.0
prompt = "a dragon"

latents= sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt]* batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode='nerf'
size=64
cameras=create_pan_cameras(size,device)
for i, latent in enumerate(latents):
  images=decode_latent_images(xm,latent,cameras,rendering_mode=render_mode)
  display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMAAAAzADMzADMzMzNmM2ZmM2aZMzNmZmZmZplmZmaZZg…

In [ ]:
from shap_e.util.notebooks import decode_latent_mesh

for i,latent in enumerate(latents):
  with open(f'leopard_{i}.ply','wb') as f:
    decode_latent_mesh(xm,latent).tri_mesh().write_ply(f)